In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# CheXNet14 

Try to predict the 14 different diseases

In [2]:
import sys
sys.path.append("../../fastai/")

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

In [4]:
# See how many devices are around
torch.cuda.device_count()
# Set it to a particular device
torch.cuda.set_device(1)
# Check which device you are on
torch.cuda.current_device()

1

In [5]:
PATH = "/home/sathiesh/Deep_Learning_Kaliy/my_projects/chest-xrays/data/"  #MMIV

In [6]:
sz=64
arch=dn121 #trent opp på ImageNet
bs = 16

In [7]:
label_csv = f'{PATH}Data_Entry_2017.csv'

# Feature engineering and data cleaning

In [8]:
df = pd.read_csv(label_csv); 
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalHeight,OriginalImagePixelSpacingx,OriginalImagePixelSpacingy,Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [9]:
df = df.drop(['Follow-up #','Patient ID', 'Patient Age', 'Patient Gender', 'View Position',
              'OriginalImageWidth', 'OriginalHeight', 'OriginalImagePixelSpacingx', 'OriginalImagePixelSpacingy', 'Unnamed: 11'],axis=1)
df.head()

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,Cardiomegaly|Emphysema
2,00000001_002.png,Cardiomegaly|Effusion
3,00000002_000.png,No Finding
4,00000003_000.png,Hernia


In [10]:
df_sorted = df.pivot_table(index='Finding Labels', aggfunc=len).sort_values('Image Index', ascending=False) 
df_sorted.head()
#TODO Hør med Alexander om noen av No Findingsene bør droppes eller? 

,Image Index
Finding Labels,
No Finding,60361
Infiltration,9547
Atelectasis,4215
Effusion,3955
Nodule,2705


In [11]:
df =df[df['Finding Labels'] != 'No Finding'] #Droppe radene med No Finding

In [12]:
df['Finding Labels'] = df['Finding Labels'].str.replace('|',' ')

In [13]:
df_sorted = df.pivot_table(index='Finding Labels', aggfunc=len).sort_values('Image Index', ascending=False) 
df_sorted

,Image Index
Finding Labels,
Infiltration,9547
Atelectasis,4215
Effusion,3955
Nodule,2705
Pneumothorax,2194
Mass,2139
Effusion Infiltration,1603
Atelectasis Infiltration,1350
Consolidation,1310


In [14]:
#df['Finding Labels'] = df['Finding Labels'].str.replace('No Finding', 'NoFinding'); 
#df['Finding Labels'] = df['Finding Labels'].str.replace('|',' ')
# df.head()

In [15]:
df.shape

(51759, 2)

In [16]:
test_image_name_list = [line.rstrip('\n') for line in open(f'{PATH}test_list.txt')]
for img_name in test_image_name_list: 
    img_row = df.loc[df['Image Index'] == img_name]
    if not img_row.empty: 
        pos = int(img_row.index.values)
        df = df.drop([pos])

In [17]:
df.shape, len(test_image_name_list)

((36024, 2), 25596)

In [18]:
df = df.reset_index(drop=True)

In [19]:
df.head()

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,Cardiomegaly Emphysema
2,00000001_002.png,Cardiomegaly Effusion
3,00000004_000.png,Mass Nodule
4,00000005_006.png,Infiltration


In [20]:
cleaned_label_path = f'{PATH}multi_labels.csv'
df.to_csv(cleaned_label_path, index = False); 

In [21]:
n = len(list(open(cleaned_label_path)))
val_idxs = get_cv_idxs(n, val_pct=0.10) 
n, len(val_idxs)

(36025, 3602)

# View data 

In [22]:
def get_data(sz): 
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on);
    return ImageClassifierData.from_csv(PATH, 'train', cleaned_label_path, test_name='test',
                                       val_idxs=val_idxs, tfms=tfms,bs=bs);

In [23]:
data = get_data(299)

In [24]:
len(data.classes), data.classes

(14,
 ['Atelectasis',
  'Cardiomegaly',
  'Consolidation',
  'Edema',
  'Effusion',
  'Emphysema',
  'Fibrosis',
  'Hernia',
  'Infiltration',
  'Mass',
  'Nodule',
  'Pleural_Thickening',
  'Pneumonia',
  'Pneumothorax'])

In [25]:
#hvordan fungerer dette? One_hot encoding
#list(zip(data.classes, y[2]))

# Training 

### Ulike størrelser
64 = metode 1:  metode2: metode3: 

In [26]:
#data = data.resize(int(sz*1.3), 'tmp') 
#Hvorfor gjør han dette steget her? (For å få det til å gå raskere (Når han henter data senere gjør han ikke dette i planet))

- Sjekk om dropout gjør loss bedre


In [37]:
learn = ConvLearner.pretrained(arch, data, ps=0.5) #, precompute=True) #HUSK å fjerne precompute

In [34]:
#learn.lr_find()
#learn.sched.plot()
learn.load('chest14')

In [166]:
learn = learn.pretrained(arch,data)
learn.save('testchest14')

In [152]:
lr = 0.01

In [30]:
#learn.fit(lr, 2); 

In [31]:
#learn.precompute = False; 
#bruk denne om dette over blir brukt:
#learn.fit(lr, 3, cycle_len=1)

In [32]:
#Hvis den over brukes, må du huske å kommentere denne ut!!!!!
#Kan også prøve precompute med denne:
learn.fit(lr, 1, cycle_len=1)
#learn.precompute = False

epoch      trn_loss   val_loss   <lambda>                      
    0      0.295224   0.277987   0.893968  



[array([0.27799]), 0.8939676373154504]

Test hva som funker best av (Marker den som brukes med X):
- unfreeze først og finne læringsraten, så kjøre fit (slik han forklarer det i easy steps). 
- finne læringsraten, så unfreeze og kjøre fit.(dette han gjør i planet datasettet). X


In [56]:
learn.unfreeze()

In [106]:
learn.pretrained(arch, data, xtra_cut=100 ,pretrained = False)

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=(1, 1))
    (mp): AdaptiveMaxPool2d(output_size=(1, 1))
  )
  (1): Flatten(
  )
  (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True)
  (3): Dropout(p=0.25)
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): ReLU()
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (7): Dropout(p=0.5)
  (8): Linear(in_features=512, out_features=14, bias=True)
  (9): Sigmoid()
)

In [48]:
#kan bare teste her fir øyeblikket

In [121]:
learn.

AttributeError: 'ConvLearner' object has no attribute 'remove'

In [34]:
#learn.lr_find()
#learn.sched.plot()

In [35]:
lrs = np.array([lr/9, lr/3, lr]) #prøv også resultatene på disse [1e-5,1e-4, 1e-3]

In [36]:
learn.fit(lrs, 2, cycle_len=1, cycle_mult=2); 

  0%|          | 0/2027 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [47]:
learn.save('chest14');

In [93]:
#learn.load('chest14')

# View the result 

In [32]:
multi_preds, y = learn.TTA(); #validerinssettet 
preds = np.mean(multi_preds,0)

In [36]:
from sklearn.metrics import fbeta_score

def f2_score(y_true, y_pred):
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    return fbeta_score(y_true, y_pred, beta=2, average='samples')

In [44]:
preds[0]

array([0.05181, 0.0045 , 0.02714, 0.00529, 0.07548, 0.01218, 0.04969, 0.0009 , 0.42122, 0.14839, 0.22003,
       0.06629, 0.01294, 0.09819], dtype=float32)

In [43]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)